In [9]:
import pandas as pd
import pickle as pkl
import os
import librosa

In [10]:
path = os.getcwd()
db = pd.read_csv(path+"/"+"file_language.csv")

In [11]:
db

,Unnamed: 0,file,language
0,0,metropolitancityofvenice_Martinamanzari_m69437...,corrupted
1,1,metropolitancityofvenice_Martatoti_m69461_a862...,Italian
2,2,guatemalacity_ivan072_m67767_a84511_audio_conv...,Latin
3,3,santiagodecompostela_Carmenvillalobos_m67573_a...,Spanish
4,4,metropolitancityofvenice_mario85_m68822_a85566...,Italian
...,...,...,...
4808,4808,metropolitancityofvenice_Giulia_m68497_a85241_...,Italian
4809,4809,metropolitancityofvenice_intrepida_m68672_a854...,Italian
4810,4810,barcelona_Ovvero_m71533_a88277_audio_converted...,Catalan
4811,4811,berlin_Wooki_m67411_a84155_audio_converted.mp3,Sundanese


In [12]:
databases = []
for language in db["language"].unique():
    if language == "corrupted":
        continue
    databases.append(db[db['language'] == language])
databases[0]

,Unnamed: 0,file,language
1,1,metropolitancityofvenice_Martatoti_m69461_a862...,Italian
4,4,metropolitancityofvenice_mario85_m68822_a85566...,Italian
7,7,metropolitancityofvenice_AnitaH_m69626_a86370_...,Italian
17,17,metropolitancityofvenice_romane_m68566_a85310_...,Italian
18,18,roma_canecanio_m66382_a83134_audio_converted.mp3,Italian
...,...,...,...
4801,4801,venice_valentinacaccamo_m68449_a85193_audio.wav,Italian
4802,4802,metropolitancityofvenice_Rossetti_m68972_a8571...,Italian
4804,4804,metropolitancityofvenice_amivenezia_m68593_a85...,Italian
4808,4808,metropolitancityofvenice_Giulia_m68497_a85241_...,Italian


# The Model

In [13]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

In [34]:
transcriptions, unsupported_languages = {}, []
non_language_databases = []
j = 0
transcriptions_file, unsupported_languages_file = "transcriptions.pkl", "unsupported_languages.pkl"
for database in databases:
    transcription_tmp, i = [], 0
    try:
        language = list(database["language"])[0].lower()
        print(language)
    except KeyError:
        non_language_databases.append(database)
        continue
    try:
        forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task="transcribe")
    except ValueError:
        unsupported_languages.append(language)
        with open(path+"/"+unsupported_languages_file,"wb") as f:
            pkl.dump(obj=unsupported_languages,file=f)
        continue

    for file in database["file"]:
        signal, sr = librosa.load(path+"/"+file,sr=16000)
        input_features = processor(signal, sampling_rate=sr, return_tensors="pt").input_features
        predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
        try:
            transcriptions[file] = transcription[0]
        except KeyError:
            transcriptions[file] = ""
        if i % 60 == 0:
            print(f"Reached iteration {i} storing results")
            
            with open(path+"/"+transcriptions_file,"wb") as f:
                pkl.dump(obj=transcriptions,file=f)
        i += 1

    print(f"Reached iteration {i} storing results")
    with open(path+"/"+transcriptions_file,"wb") as f:
        pkl.dump(obj=transcriptions,file=f)

italian
latin
spanish
polish
catalan
galician
bosnian
belarusian
german
sundanese
norwegian nynorsk
portuguese
welsh
javanese
macedonian
slovenian
esperanto
greek
english
basque
occitan
Reached iteration 0 storing results
Reached iteration 22 storing results
hawaiian
Reached iteration 0 storing results
Reached iteration 13 storing results
waray
yoruba
Reached iteration 0 storing results
Reached iteration 8 storing results
malay
Reached iteration 0 storing results
Reached iteration 23 storing results
norwegian
Reached iteration 0 storing results
Reached iteration 16 storing results
vietnamese
Reached iteration 0 storing results
Reached iteration 8 storing results
somali
Reached iteration 0 storing results
Reached iteration 5 storing results
dutch
Reached iteration 0 storing results
Reached iteration 5 storing results
luxembourgish
Reached iteration 0 storing results
Reached iteration 10 storing results
burmese
Reached iteration 0 storing results
Reached iteration 10 storing results
taga